# RandomSearch using Keras-Tuner

In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("churn_modelling.csv")

In [4]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
X=df.iloc[:,3:13]
y=df.iloc[:,13]

In [6]:
geography=pd.get_dummies(X['Geography'], drop_first=True)
gender=pd.get_dummies(X['Gender'], drop_first=True)

In [7]:
X=pd.concat([X,geography,gender],axis=1)

In [8]:
X=X.drop(['Geography','Gender'],axis=1)

In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense

def build_model(hp):
    model=Sequential()

    for i in range(hp.Int('num_layers',2,20)):
        model.add(Dense(units=hp.Int("units_"+str(i),min_value=32,max_value=512,step=32),activation="relu"))

    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="Adamax",loss="binary_crossentropy",metrics=["accuracy"])
    return model

In [10]:
from keras_tuner import RandomSearch

tuner=RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=5,
    executions_per_trial=3,
    directory="hyperparameter",
    project_name="Hyperparameter Tuning"
)


In [11]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0)

In [16]:
tuner.search(X_train,y_train,epochs=5, validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 58s]
val_accuracy: 0.7929999828338623

Best val_accuracy So Far: 0.7931111057599386
Total elapsed time: 00h 02m 43s


In [18]:
tuner.results_summary()

Results summary
Results in hyperparameter\Hyperparameter Tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
num_layers: 4
units_0: 64
units_1: 352
units_2: 32
units_3: 32
Score: 0.7931111057599386

Trial 1 summary
Hyperparameters:
num_layers: 6
units_0: 352
units_1: 352
units_2: 384
units_3: 96
units_4: 32
units_5: 32
Score: 0.7929999828338623

Trial 2 summary
Hyperparameters:
num_layers: 12
units_0: 320
units_1: 32
units_2: 416
units_3: 64
units_4: 96
units_5: 416
units_6: 32
units_7: 32
units_8: 32
units_9: 32
units_10: 32
units_11: 32
Score: 0.7929999828338623

Trial 4 summary
Hyperparameters:
num_layers: 15
units_0: 32
units_1: 192
units_2: 288
units_3: 480
units_4: 96
units_5: 480
units_6: 480
units_7: 512
units_8: 96
units_9: 160
units_10: 192
units_11: 384
units_12: 32
units_13: 32
units_14: 32
Score: 0.7929999828338623

Trial 3 summary
Hyperparameters:
num_layers: 3
units_0: 192
units_1: 224
units_2: 480
units_3: 128


# GridSearchCV

In [36]:
def create_model(layers,activation,input_dim, **kwargs):
    
    model=Sequential()

    for i, nodes in enumerate(layers):

        if(i==0):
            model.add(Dense(nodes,input_dim=input_dim))
            model.add(Activation(activation))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
    model.add(Dense(units=1, kernel_initializer='glorot_uniform',activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])

    return model

            
    

In [37]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

from keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization
from tensorflow.keras.activations import relu, sigmoid


model = KerasClassifier(
    build_fn=create_model,
    input_dim=X_train.shape[1],
    verbose=0,
    layers=None,
    activation=None,
)

In [38]:
layers=[[20],[40,20],[45,30,15]]
activations=['sigmoid','relu']

param_grid=dict(layers=layers, activation=activations, batch_size=[128,256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)

grid_result= grid.fit(X_train, y_train)

C:\Users\HP\anaconda3\envs\ann_hyperparameter\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\HP\anaconda3\envs\ann_hyperparameter\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\HP\anaconda3\envs\ann_hyperparameter\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\HP\anaconda3\envs\ann_hyperparameter\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pa

C:\Users\HP\anaconda3\envs\ann_hyperparameter\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\HP\anaconda3\envs\ann_hyperparameter\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


C:\Users\HP\anaconda3\envs\ann_hyperparameter\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\HP\anaconda3\envs\ann_hyperparameter\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\HP\anaconda3\envs\ann_hyperparameter\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\HP\anaconda3\envs\ann_hyperparameter\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pa

In [40]:
print(grid_result.best_score_,grid_result.best_params_)

0.7977142857142857 {'activation': 'sigmoid', 'batch_size': 128, 'epochs': 30, 'layers': [20]}
